In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Bonneville Dam Data Source

http://www.cbr.washington.edu/dart/query/adult_annual_sum


In [2]:
# Read in Bonneville Dam Spring Chinook Counts
bonneville_file = "Resources/Chinook_Bonneville_Counts.csv"

# Grab only the columns we want
bonneville_salmon_df = pd.read_csv(bonneville_file, usecols=["Project","Year","Dates", "Chinook"])

# Make sure we only get 2000 to 2020
bonneville_salmon_df =  bonneville_salmon_df[ 
    (bonneville_salmon_df["Year"] > 1999) & 
    (bonneville_salmon_df["Year"] < 2021)
]

# Clean up data types
bonneville_salmon_df["Year"] = bonneville_salmon_df["Year"].astype('int')
bonneville_salmon_df["Chinook"] = bonneville_salmon_df["Chinook"].astype('int')

bonneville_salmon_df

,Project,Year,Dates,Chinook
62,Bonneville,2000,3/15 - 5/31,178302
63,Bonneville,2001,3/15 - 5/31,391367
64,Bonneville,2002,3/15 - 5/31,268813
65,Bonneville,2003,3/15 - 5/31,192010
66,Bonneville,2004,3/15 - 5/31,170152
67,Bonneville,2005,3/15 - 5/31,74038
68,Bonneville,2006,3/15 - 5/31,96456
69,Bonneville,2007,3/15 - 5/31,66624
70,Bonneville,2008,3/15 - 5/31,125543
71,Bonneville,2009,3/15 - 5/31,114525


## Bonneville Below Data Source

https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=14128600

mapper: https://maps.waterdata.usgs.gov/mapper/index.html


In [3]:
#read in gage data for Stevenson
stevenson_file = "Resources/Columbia_bonne_stevenson.csv"
Columbia_bonne_stevenson_df = pd.read_csv(stevenson_file, sep="\t")
Columbia_bonne_stevenson_df

,agency_cd,site_no,datetime,Gage_Max,Status_Gage_Max,Gage_min,Status_gage_min,Gage_Mean,Status_Gage_Mean
0,USGS,14128600,2008-10-01,76.23,A,75.93,A,76.05,A
1,USGS,14128600,2008-10-02,76.33,A,76.03,A,76.21,A
2,USGS,14128600,2008-10-03,76.53,A,76.03,A,76.33,A
3,USGS,14128600,2008-10-04,76.33,A,76.03,A,76.11,A
4,USGS,14128600,2008-10-05,76.43,A,76.13,A,76.31,A
...,...,...,...,...,...,...,...,...,...
4412,USGS,14128600,2020-10-30,73.59,P,72.70,P,72.95,P
4413,USGS,14128600,2020-10-31,75.11,P,73.59,P,74.44,P
4414,USGS,14128600,2020-11-01,75.13,P,74.30,P,74.58,P
4415,USGS,14128600,2020-11-02,74.67,P,74.07,P,74.30,P


In [4]:
#find rows with null values
Columbia_bonne_stevenson_df[Columbia_bonne_stevenson_df.isnull().values.any(axis=1)]

,agency_cd,site_no,datetime,Gage_Max,Status_Gage_Max,Gage_min,Status_gage_min,Gage_Mean,Status_Gage_Mean
4094,USGS,14128600,2019-12-17,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#drop unnecessary columns
cbs_clean_df = Columbia_bonne_stevenson_df.dropna()
cbs_clean_df = cbs_clean_df.reset_index(drop=True)
cbs_clean_df

,agency_cd,site_no,datetime,Gage_Max,Status_Gage_Max,Gage_min,Status_gage_min,Gage_Mean,Status_Gage_Mean
0,USGS,14128600,2008-10-01,76.23,A,75.93,A,76.05,A
1,USGS,14128600,2008-10-02,76.33,A,76.03,A,76.21,A
2,USGS,14128600,2008-10-03,76.53,A,76.03,A,76.33,A
3,USGS,14128600,2008-10-04,76.33,A,76.03,A,76.11,A
4,USGS,14128600,2008-10-05,76.43,A,76.13,A,76.31,A
...,...,...,...,...,...,...,...,...,...
4411,USGS,14128600,2020-10-30,73.59,P,72.70,P,72.95,P
4412,USGS,14128600,2020-10-31,75.11,P,73.59,P,74.44,P
4413,USGS,14128600,2020-11-01,75.13,P,74.30,P,74.58,P
4414,USGS,14128600,2020-11-02,74.67,P,74.07,P,74.30,P


In [6]:
#Get rid of columns you don't need
cbs_reduce_df = cbs_clean_df.drop(columns=['agency_cd', 'site_no', 'Status_Gage_Max', 'Status_gage_min', 'Status_Gage_Mean'])
cbs_reduce_df = cbs_reduce_df.reset_index(drop=True)
cbs_reduce_df.head()

,datetime,Gage_Max,Gage_min,Gage_Mean
0,2008-10-01,76.23,75.93,76.05
1,2008-10-02,76.33,76.03,76.21
2,2008-10-03,76.53,76.03,76.33
3,2008-10-04,76.33,76.03,76.11
4,2008-10-05,76.43,76.13,76.31


In [7]:
#Split the data into three columns
cbs_reduce_df[['Year','Month','Day']] = cbs_reduce_df['datetime'].str.split('-', expand=True)
cbs_reduce_df = cbs_reduce_df.drop(columns=['datetime'])
cbs_reduce_df = cbs_reduce_df.reset_index(drop=True)
cbs_reduce_df.head()

,Gage_Max,Gage_min,Gage_Mean,Year,Month,Day
0,76.23,75.93,76.05,2008,10,01
1,76.33,76.03,76.21,2008,10,02
2,76.53,76.03,76.33,2008,10,03
3,76.33,76.03,76.11,2008,10,04
4,76.43,76.13,76.31,2008,10,05


In [8]:
#changed the day, month, year columns to be numbers
cbs_reduce_df['Month'] = cbs_reduce_df['Month'].astype(int)
cbs_reduce_df['Year'] = cbs_reduce_df['Year'].astype(int)
cbs_reduce_df['Day'] = cbs_reduce_df['Day'].astype(int)
cbs_reduce_df.dtypes

Gage_Max       float64
Gage_min       float64
Gage_Mean      float64
Year             int32
Month            int32
Day              int32
dtype: object

In [9]:
#only bring in Mar, Apr, May data
cbs_final= cbs_reduce_df[(cbs_reduce_df['Month'] >= 3) & (cbs_reduce_df['Month'] <= 5)]
cbs_final.head()

,Gage_Max,Gage_min,Gage_Mean,Year,Month,Day
151,76.06,74.89,75.39,2009,3,1
152,76.24,74.96,75.83,2009,3,2
153,76.48,74.97,75.65,2009,3,3
154,76.70,76.29,76.47,2009,3,4
155,76.93,76.20,76.57,2009,3,5


In [10]:
#Average the above data by year to get the gage average by year in the spring
cbs_final2 = cbs_final.groupby("Year").mean()
cbs_final2.head()

,Gage_Max,Gage_min,Gage_Mean,Month,Day
Year,,,,,
2009,76.630652,75.384457,75.929348,4.0,15.836957
2010,76.328913,75.411957,75.806304,4.0,15.836957
2011,76.614457,75.411304,75.953152,4.0,15.836957
2012,76.822826,75.513696,76.115870,4.0,15.836957
2013,76.132500,75.000326,75.501739,4.0,15.836957
